In [2]:
import pandas as pd
import numpy as np

# INPUT
output_url = '../output/testing.xlsx'

data_url = '../data/ORIGIN.xlsx'
data_sheetname = 'Service-T3'
data_header_starter = 13

cd_col = 'CD no'
invoice_col = 'Invoice no'
bill_col = 'BILL'
declaration_col = 'Date of declaration'
clearance_col = 'Custom Clearance'
other_col = 'Others'
c_o_col = 'C/O'
pay_on_behalf_col = 'Pay on behalf'

vat = 0.08
# GET FORM

data = pd.read_excel(data_url, sheet_name=data_sheetname, header=(data_header_starter-1), converters={cd_col: str, invoice_col: str})
data = data.loc[~pd.isna(data[invoice_col]) & ~pd.isna(data[bill_col])]
data[declaration_col] = data[declaration_col].ffill()
data[bill_col] = data[bill_col].ffill()
data[cd_col] = data[cd_col].ffill()
data[invoice_col] = data[invoice_col].ffill()

grouped = data.groupby(bill_col, sort=False)

writer = pd.ExcelWriter(output_url, engine='xlsxwriter')
wookbook = writer.book
wookbook.formats[0].set_font_size(12)

# STYLE CONFIG
company_name_format = wookbook.add_format({
    'font_size': 14,
})
merge_format = wookbook.add_format({
    'bold': 1,
    'align': 'center',
    'valign': 'vcenter',
    # 'fg_color': 'yellow'
})
debit_note_format = wookbook.add_format({
    'font_size': 22,
    'align': 'center',
    'bold': True
})
title_format = wookbook.add_format({
    'font_size': 22,
    'align': 'center',
})
bold = wookbook.add_format({
    'bold': True,
    'font_size': 12
})
center_format = wookbook.add_format(
    {'bold': True,
    'border': 1,
    'align': 'center', 
    'valign': 'vcenter'}
)
table = wookbook.add_format({
    'border': 1,
})

num_format = wookbook.add_format({'num_format': '#,##0.00'})


for name, group in grouped:
    current_row = 0
    pd.DataFrame().to_excel(writer, sheet_name=name, index=False, header=False, startrow=current_row)
    worksheet = writer.sheets[name]

    # ------------ HEADER --------------------
    worksheet.write(f'A{str(current_row + 1)}', 'UNI CONSULTING CO., LTD', company_name_format)
    current_row += 1

    worksheet.write(f'A{str(current_row + 1)}', '113A Đường 109, Khu Phố 5, Phường Phước Long B, Thành phố Thủ Đức, Thành phố Hồ Chí Minh, Việt Nam')
    current_row += 1
    current_row += 1

    # BREAK
    worksheet.merge_range(f'B{str(current_row + 1)}:F{str(current_row + 3)}', '________________________________________________________________________________________________________________________________________', merge_format)
    current_row += 3

    # TITLE
    worksheet.merge_range(f'B{str(current_row + 1)}:F{str(current_row + 1)}', 'DEBIT NOTE', debit_note_format)
    current_row += 1
    worksheet.merge_range(f'B{str(current_row + 1)}:F{str(current_row + 1)}', 'CLEARANCE SERVICE AND PAY ON BEHAFL', title_format)
    current_row += 1

    # TO ASIA
    worksheet.write(f'B{str(current_row + 1)}', 'To:', bold)
    worksheet.write(f'C{str(current_row + 1)}', 'ASIA LCD SERVICE', bold)
    current_row += 1

    # HBL
    worksheet.write(f'B{str(current_row + 1)}', 'HBL:', bold)
    worksheet.write(f'C{str(current_row + 1)}', name, bold)
    worksheet.write(f'E{str(current_row + 1)}', 'Invoice No.:', bold)

    invoices = group['Invoice no'].unique()
    invoices_df = pd.DataFrame(invoices)
    invoices_df.to_excel(writer, sheet_name=name, index=False, header=False, startrow=(current_row), startcol=5)

    current_row += len(invoices)

    # ------------ CDS ---------------
    worksheet.write(f'B{str(current_row + 1)}', 'CDS:', bold)
    worksheet.write(f'E{str(current_row + 1)}', 'POL/POD:', bold)
    cds = group['CD no'].unique()
    cds_df = pd.DataFrame(cds)
    cds_df.to_excel(writer, sheet_name=name, index=False, header=False, startrow=current_row, startcol=2)
    current_row += len(cds)

    # ------------ TERM AND TRUCK -------------
    worksheet.write(f'B{str(current_row + 1)}', 'TERM:', bold)
    worksheet.write(f'E{str(current_row + 1)}', 'TRUCK', bold)
    current_row += 1

    # ------------ DATE DECLARATION -------------
    worksheet.write(f'B{str(current_row + 1)}', 'Date of declaration', bold)
    worksheet.write(f'E{str(current_row + 1)}', 'GWT (kg):', bold)
    declaration_dates = group[declaration_col].unique()
    declaration_dates = [dt for dt in declaration_dates if not pd.isna(dt)]
    declaration_dates = [pd.to_datetime(str(x)).strftime('%d-%m-%Y') for x in declaration_dates]
    declaration_dates_df = pd.DataFrame(declaration_dates)
    declaration_dates_df.to_excel(writer, sheet_name=name, index=False, header=False, startrow=current_row, startcol=2)
    current_row += len(declaration_dates)
    current_row += 1

    # --- SERVICE COST ----
    worksheet.write(f'B{str(current_row + 1)}', 'SERVICE COST (VND)')
    current_row += 1

    start_table = str(current_row + 1)
    worksheet.write(f'B{str(current_row + 1)}', 'No.', center_format)
    worksheet.write(f'C{str(current_row + 1)}', 'Freight', center_format)
    worksheet.write(f'D{str(current_row + 1)}', 'Rate', center_format)
    worksheet.write(f'E{str(current_row + 1)}', 'VAT', center_format)
    worksheet.write(f'F{str(current_row + 1)}', 'Amount', center_format)
    worksheet.write(f'G{str(current_row + 1)}', 'Note', center_format)
    current_row += 1

    clearance = group[clearance_col].sum()
    other = group[other_col].sum()
    c_o = group[c_o_col].sum()
    pay_on_behalf = group[pay_on_behalf_col].sum()

    worksheet.write(f'B{str(current_row + 1)}', 1)
    worksheet.write(f'C{str(current_row + 1)}', 'Customs Clearance')
    worksheet.write(f'D{str(current_row + 1)}', clearance)
    worksheet.write(f'E{str(current_row + 1)}', clearance * vat)
    worksheet.write(f'F{str(current_row + 1)}', clearance*(1+vat))
    current_row += 1

    worksheet.write(f'B{str(current_row + 1)}', 2)
    worksheet.write(f'C{str(current_row + 1)}', 'Other')
    worksheet.write(f'D{str(current_row + 1)}', other)
    worksheet.write(f'E{str(current_row + 1)}', other*vat)
    worksheet.write(f'F{str(current_row + 1)}', other*(1+vat))
    current_row += 1

    worksheet.write(f'B{str(current_row + 1)}', 3)
    worksheet.write(f'C{str(current_row + 1)}', 'C/O')
    worksheet.write(f'D{str(current_row + 1)}', c_o)
    worksheet.write(f'E{str(current_row + 1)}', c_o*vat)
    worksheet.write(f'F{str(current_row + 1)}', c_o*(1+vat))
    current_row += 1

    worksheet.write(f'B{str(current_row + 1)}', 4)
    worksheet.write(f'C{str(current_row + 1)}', 'Pay on behalf')
    worksheet.write(f'D{str(current_row + 1)}', pay_on_behalf)
    # worksheet.write(f'E{str(current_row + 1)}', pay_on_behalf*vat)
    worksheet.write(f'F{str(current_row + 1)}', pay_on_behalf)
    current_row += 1

    worksheet.write(f'C{str(current_row + 1)}', 'TOTAL AMOUNT (VND)', bold)
    worksheet.write(
        f'F{str(current_row + 1)}', 
        clearance*(1+vat) + other*(1+vat) + c_o*(1+vat) + pay_on_behalf, bold
    )
    current_row += 1
    end_table = current_row

    # FORMAT TABLE
    worksheet.conditional_format(f'B{start_table}:G{end_table}', 
        {
            'type': 'no_blanks',
            'format': table
        }
    )
    worksheet.conditional_format(f'B{start_table}:G{end_table}', 
        {
            'type': 'blanks',
            'format': table
        }
    )
    worksheet.conditional_format(f'B{start_table}:G{end_table}', {
                                        'type': 'cell',
                                        'criteria': '>',
                                        'value': 999,
                                        'format': num_format})

    current_row += 1
    worksheet.write(f'B{str(current_row + 1)}', 'Telegraphic Banking transfer:', bold)
    current_row += 2

    worksheet.write(f'B{str(current_row + 1)}', 'Beneficial Name')
    worksheet.write(f'D{str(current_row + 1)}', 'CONG TY TNHH TU VAN UNI')
    current_row += 1

    worksheet.write(f'B{str(current_row + 1)}', 'Bank Name')
    worksheet.write(f'D{str(current_row + 1)}', 'SHINHAN BANK VIETNAM - BIEN HOA BRANCH')
    current_row += 1
    
    worksheet.write(f'B{str(current_row + 1)}', 'Account No (VND)')
    worksheet.write(f'D{str(current_row + 1)}', '700-019-155994')
    worksheet.write(f'H{str(current_row + 1)}', '')
    current_row += 1

    worksheet.set_column(1, 2, 16)
    worksheet.set_column(3, 3, 13)
    worksheet.set_column(4, 4, 11)
    worksheet.set_column(5, 5, 16)
    # worksheet.set_landscape()
    worksheet.fit_to_pages(1, 1)

writer.save()
writer.close()

d:\Projects\Excel-to-SQL-snippets\env\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
